In [1]:
import pandas as pd
from konlpy.tag import Okt
import glob
import re
from bs4 import BeautifulSoup
import requests
import random
import time
import pandas as pd
from tqdm import tqdm

In [2]:
file='../기업프로젝트/output.csv'
output=pd.read_csv(file)
output

,Unnamed: 0.1,Unnamed: 0,뉴스 식별자,일자,언론사,기고자,제목,통합 분류1,통합 분류2,통합 분류3,...,사건/사고 분류3,인물,위치,기관,키워드,특성추출(가중치순 상위 50개),본문,URL,분석제외 여부,요약
0,0,1,1.100611e+06,20230115,서울신문,홍지민,"하윤기, 덩크 9방에 생애 첫 올스타전 MVP",스포츠>축구,스포츠>농구_배구,스포츠>올림픽_아시안게임,...,NaN,NaN,"울산,전주,서울,준결승전,경기,영광,창원,아반도,필리핀,아시아,안양,대구,문성곤","한국가스공사,KGC,SK,kt,KCC,랜드리,LG,렌즈 아반도,아셈,삼성,현대모비스...","하윤기,덩크,생애,올스타전,MVP,이대성,제압,허웅,전성현,3점,분전,허웅,생애,3...","허웅,덩크,올스타전,3점,하윤기,이대성,kt,전성현,외국인,팬투표,덩크슛,9개,서울...",팀 이대성 122 117로 팀 허웅 제압 전성현 3점슛 9개 분전허웅 생애 첫 ...,http://www.seoul.co.kr/news/newsView.php?id=20...,NaN,팀 이대성 122 117로 팀 허 웅 제압 전 성현 3점 슛 9개 분전 허 웅 생애...
1,1,3,1.100611e+06,20230115,서울신문,신진호,"장제원, 나경원 향해 “제2의 유승민 되지 말기를”",정치>국회_정당,정치>선거,정치>청와대,...,NaN,"유승민,김기현,나경원,장제원,전,유일한,장,조원진,윤석열,유 전,마키아벨리","을,동구,파동,대구","친박,정부,친박계,파동,대한민국,진박감별사,새누리당,페이스북,국민의힘","장제원,나경원,제2,유승민,제2,진박감별사,나경원,비판,응수,나경원,국민,의원,출마...","감별사,새누리당,유승민,박근혜,나경원,마키아벨리,친박,진박,진박감별사,페이스북,장제...",제2의 진박감별사가 당 쥐락펴락 나경원 비판에 응수 국민의힘 장제원 의원 나...,http://www.seoul.co.kr/news/newsView.php?id=20...,NaN,제 2의 진 박 감별사가 당 쥐락펴락 나 경원 비판에 응수 국민의 힘 장 제원 의원...
2,2,5,1.500601e+06,20230115,매일신문,이수현,[대구 국회의원 평가 여론조사] 북구갑 양금희 당직 맡아 국회평가 높아,정치>국회_정당,NaN,NaN,...,NaN,"양금희,윤석열,홍준표","대구,대구시,대야,부여,북구,북구갑,선봉","국민의힘,국회,더불어민주당,매일신문,정의당,중앙선거여론조사심의위원회,한길리서치,행정안전부","북구갑,양금희,당직,국회평가,59%,북구갑,주민,59%,대통령,국정,수행,긍정,시장...","북구갑,대구,양금희,응답자,ars,한길리서치,홈페이지,행정안전부,18세,여론조사,부...",양금희 국민의힘 의원 대구 북구갑 . 대구...,https://news.imaeil.com/page/view/202301152129...,NaN,이 한길 리서치에 의뢰해 지난 9~11 일 대구 북구 갑에 거주하는 만 18세 이상...
3,3,7,1.500601e+06,20230115,매일신문,이수현,[대구 국회의원 평가 여론조사] 서구 김상훈 3선 의정활동 평가 무난,지역>대구,지역>대전,정치>국회_정당,...,NaN,"김,김상훈,윤석열,홍준표","대구,대구시,부여,서구","국민의힘,더불어민주당,매일신문,정의당,중앙선거여론조사심의위원회,한길리서치,행정안전부","서구,김상훈,3선,무난,의정,활동,평가,대통령,국정수행,62.1%,30.4%,긍정,...","대구,김상훈,ars,한길리서치,홈페이지,행정안전부,응답자,3선,홍준표,부여,정의당,...",김상훈 국민의힘 의원 대구 서구 . 대구 ...,https://news.imaeil.com/page/view/202301152122...,NaN,이 한길 리서치에 의뢰해 지난 9~11 일 대구 서구에 거주하는 만 18세 이상 남...
4,4,9,1.500601e+06,20230115,매일신문,이수현,[대구 국회의원 평가 여론조사] 동구갑 류성걸 의원,지역>대구,지역>대전,정치>국회_정당,...,NaN,"류성걸,윤석열,홍준표","대구,대구시,동구,동구갑,부여","국민의힘,더불어민주당,매일신문,정의당,중앙선거여론조사심의위원회,한길리서치,행정안전부","의원,동구갑,류성걸,재선,평균치,의원,류성걸,국민의힘,전반적,의정,활동,주민들,대구...","동구갑,류성걸,대구,국민의힘,동구,ars,한길리서치,홈페이지,행정안전부,18세,지역...",류성걸 국민의힘 의원 대구 동구갑 . 류성...,https://news.imaeil.com/page/view/202301152115...,NaN,이 한길 리서치에 의뢰해 지난 9~11 일 대구 동구 갑에 거주하는 만 18세 이상...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39941,39941,80036,2.100311e+06,20230821,서울경제,서재원 기자,1년 전 銀을 金으로 우상혁 '위대한 도전',스포츠>올림픽_아시안게임,스포츠>축구,스포츠>월드컵,...,NaN,"김현섭,우상혁","독일,미국,부다페스트,도하,일본,유진 세계선수권대회,한국,세계선수권,뮌헨,텍사스,쿠...","아카마쓰,용인시청","1년,우상혁,도전,세계,선수권,높이뛰기,우승,도전,바르심,무타즈,에사,32,저본,해...","우상혁,한국,세계선수권,미국,해리슨,높이뛰기,선수권,바르심,런던,다이아몬드리그,유진...",우상혁이 20일 2023 부다페스트 세계육상선수권 남자 높이뛰기 예선에서 바를 넘고...,http://www.sedaily.com/NewsView/29THRF514Y,NaN,우상 혁은 20일 이하 한국 시간 헝가리 부 다 페스트 국립 육상 경기장에서 열린 ...
39942,39942,80038,2.100311e+06,20230820,서울경제,여론독자부,[인사] 중소벤처기업부 외,지역>대구,경제>취업_창업,지역>전북,...,NaN,"이용주,이대희,박금철,김진명,이현조,조만희,이형철,김현곤,원영준,강영규,김언성,박정현","대구,경북지방중소벤처","△공공정책국장,△중앙노동위원회,기획재정부,△재정관리국장,고용노동부,중소벤처기업부","중소벤처기업부,전보,중소,벤처,기업부,실장,중소기업정책실장,전보,이대희,소상공인,정...","정책관,김현곤,박금철,조만희,이용주,중소벤처기업부,김언성,김진명,이형철,이대희,원영...",중소벤처기업부 실장급 전보 중소기업정책실장 이대희 소상공인정책실장 원영준 국...,http://www.sedaily.com/NewsView/29THBVO7B0,예외,중소 벤처기업 부 실장 급 전보 중소기업정책실장 이 대희 소 상공인정책실장 원 영준...
39943,39943,80040,2.100311e+06,20230820,서울경제,김경훈 기자,"'명예훼손 실형' 김용호, 조국 김건모 전처에 사과 ""깊이 반성""",사회>미디어,사회>사건_사고,NaN,...,NaN,"김,조,여배우,김건모,강용석,조국,김용호",대구,"유튜브,가로세로연구소,KNL,법무부,법원,유튜버","명예,훼손,실형,김용호,조국,김건모,전처,사과,반성,연예,조국,법무부,장관,가수,김...","김씨,김건모,김용호,사생활,장씨,연구소,가로세로,유튜브,가로세로연구소,유튜버,연예부...",유튜버 김용호씨. 유튜브 채널 강용석 나이트 라이브 캡처연예 출신 유튜...,http://www.sedaily.com/NewsView/29THC9PDCP,NaN,라는 제목의 영상을 통해 2019년 8월 김용호 연예부장 채널에서 방송한 조국 전 ...
39944,39944,80042,2.100311e+06,20230820,서울경제,이승령 기자,이러니 치안 불안할 수 밖에 현장 누빌 순경 1.4만명 부족,지역>광주,지역>부산,지역>울산,...,NaN,"순경,정우택","경기북,경기남부,서울,부산,대구,인천,경남,지구대","서울경찰청,행정안전위원회,국회,경찰청,경창청,국민의힘,비간부","치안,불안,현장,1.4,누빌,순경,1.4,부족,경감,경위,경정,간부직,정원,사건,시...","경찰관,경찰청,퇴직자,서울,시도경찰청,치안현장,경찰관들,정우택,3만,6급,국민의힘,...",윤희근 경

In [3]:
output=output[['본문', '요약']]

In [27]:
output['본문'][3974]

'삼보모터스  삼보문화재단 왼쪽 이재하 회장 은 대구 문화예술발전을 위해 연 3천만원씩 총 3억원을 대구문화예술진흥원에 기부약정한다. 대구문화예술진흥원 제공                    삼보모터스  삼보문화재단 회장 이재하 은 대구 문화예술 발전을 위해 연 3천만원씩 총 3억원을 대구문화예술진흥원 이하 문예진흥원 에 기부 약정한다.삼보모터스는 지역을 대표하는 기업 중 하나로 2015년 삼보문화재단을 설립해 지역의 예술인을 지원 육성하고 전통문화 계승과 발전을 위해 사회공헌과 나눔을 실천하는 기업이다. 청년 시절 미술 교사였던 이재하 회장은 6일 출범한  대구메세나협의회 의 회장으로 추대돼 지역 메세나 활성화와 문화예술 발전을 위해 앞장서고 있다.삼보모터스 삼보문화재단은 기부 약정과 더불어 삼보미술상 제정 의사를 밝혔다. 대구문예진흥원은 미술 분야에서 창작활동에 전념하고 예술적 성과를 인정받은 훌륭한 지역의 원로 작가 1명과 예술적 잠재력과 발전 가능성이 큰 청년 작가 2명을 선정해 삼보미술상을 시상할 계획이다. 삼보미술상에 선정되면 상금과 함께 이듬해에 대구문화예술회관에서 기념전시를 개최할 수 있다.'

In [28]:
output['요약'][3974]

'삼보 모터스는 지역을 대표하는 기업 중 하나로 2015년 삼보문화재단을 설립해 지역의 예술인을 지원 육성하고 전통문화 계승과 발전을 위해 사회 공헌과 나눔을 실천하는 기업이다. 대구 문예 진흥원은 미술 분야에서 창작활동에 전념하고 예술적 성과를 인정받은 훌륭한 지역의 원로 작가 1명과 예술적 잠재력과 발전 가능성이 큰 청년 작가 2명을 선정해 삼보 미술상을 시상할 계획이다.'

In [13]:
sorted_output = output.assign(body_length=output['본문'].apply(len)).sort_values('body_length')



                 본문              요약  body_length
4896         대구 김승재         대구 김 승재            6
21449        4 2 키움             NaN            6
12977     울산 1 0 강원       울산 1 0 강원            9
13909     울산 2 1 광주       울산 2 1 광주            9
14215     전북 1 2 대전      전 북 1 2 대전            9
5783      자료  키노라이츠       자료 키 노라이츠            9
14400   티켓은 전석 초대다.     티켓은 전석 초대다.           11
2304   부산 김상윤   김민기   부산 김상 윤 김 민 기           12
30301  여기힙해 레터 구독하기  여기 힙 해 레터 구독하기           12
9341   사진  뮤지엄한미 삼청   사진 뮤 지엄한 미 삼청           12


In [26]:
# Take the top 10 rows with the least content
top_10_rows = sorted_output.head(10000)

print(top_10_rows)

                                                      본문  \
4896                                              대구 김승재   
21449                                             4 2 키움   
12977                                          울산 1 0 강원   
13909                                          울산 2 1 광주   
14215                                          전북 1 2 대전   
...                                                  ...   
18158  1.  카눈  북한으로 아침까지 중부 영향권 태풍 카눈이 오늘 11일  새벽 1시 ...   
36169  달성군청소년센터 해외문화체험에 참여한 관내 고등학생 16명이 장가계 황룡동굴 입구에...   
21567  7월 11일 화요일  오후 날씨입니다.이 시간 현재 전국 대부분 지방에 호우주의보가...   
28283  클릭하시면 더 큰이미지를 보실 수 있습니다 사진은 추석 당일인 지난 9월 29일 오...   
3974   삼보모터스  삼보문화재단 왼쪽 이재하 회장 은 대구 문화예술발전을 위해 연 3천만원...   

                                                      요약  body_length  
4896                                             대구 김 승재            6  
21449                                                NaN            6  
12977                                          울산 1 0 강원       

In [4]:
file='../기업프로젝트/finalNews.csv'
df=pd.read_csv(file)
df

,Unnamed: 0,뉴스 식별자,일자,언론사,기고자,제목,통합 분류1,통합 분류2,통합 분류3,사건/사고 분류1,사건/사고 분류2,사건/사고 분류3,인물,위치,기관,키워드,특성추출(가중치순 상위 50개),본문,URL,분석제외 여부
0,1,1.100611e+06,20230115,서울신문,홍지민,"하윤기, 덩크 9방에 생애 첫 올스타전 MVP",스포츠>축구,스포츠>농구_배구,스포츠>올림픽_아시안게임,사고>스포츠사고,NaN,NaN,NaN,"울산,전주,서울,준결승전,경기,영광,창원,아반도,필리핀,아시아,안양,대구,문성곤","한국가스공사,KGC,SK,kt,KCC,랜드리,LG,렌즈 아반도,아셈,삼성,현대모비스...","하윤기,덩크,생애,올스타전,MVP,이대성,제압,허웅,전성현,3점,분전,허웅,생애,3...","허웅,덩크,올스타전,3점,하윤기,이대성,kt,전성현,외국인,팬투표,덩크슛,9개,서울...",팀 이대성 122 117로 팀 허웅 제압 전성현 3점슛 9개 분전허웅 생애 첫 ...,http://www.seoul.co.kr/news/newsView.php?id=20...,NaN
1,3,1.100611e+06,20230115,서울신문,신진호,"장제원, 나경원 향해 “제2의 유승민 되지 말기를”",정치>국회_정당,정치>선거,정치>청와대,NaN,NaN,NaN,"유승민,김기현,나경원,장제원,전,유일한,장,조원진,윤석열,유 전,마키아벨리","을,동구,파동,대구","친박,정부,친박계,파동,대한민국,진박감별사,새누리당,페이스북,국민의힘","장제원,나경원,제2,유승민,제2,진박감별사,나경원,비판,응수,나경원,국민,의원,출마...","감별사,새누리당,유승민,박근혜,나경원,마키아벨리,친박,진박,진박감별사,페이스북,장제...",제2의 진박감별사가 당 쥐락펴락 나경원 비판에 응수 국민의힘 장제원 의원 나...,http://www.seoul.co.kr/news/newsView.php?id=20...,NaN
2,5,1.500601e+06,20230115,매일신문,이수현,[대구 국회의원 평가 여론조사] 북구갑 양금희 당직 맡아 국회평가 높아,정치>국회_정당,NaN,NaN,NaN,NaN,NaN,"양금희,윤석열,홍준표","대구,대구시,대야,부여,북구,북구갑,선봉","국민의힘,국회,더불어민주당,매일신문,정의당,중앙선거여론조사심의위원회,한길리서치,행정안전부","북구갑,양금희,당직,국회평가,59%,북구갑,주민,59%,대통령,국정,수행,긍정,시장...","북구갑,대구,양금희,응답자,ars,한길리서치,홈페이지,행정안전부,18세,여론조사,부...",양금희 국민의힘 의원 대구 북구갑 . 대구...,https://news.imaeil.com/page/view/202301152129...,NaN
3,7,1.500601e+06,20230115,매일신문,이수현,[대구 국회의원 평가 여론조사] 서구 김상훈 3선 의정활동 평가 무난,지역>대구,지역>대전,정치>국회_정당,NaN,NaN,NaN,"김,김상훈,윤석열,홍준표","대구,대구시,부여,서구","국민의힘,더불어민주당,매일신문,정의당,중앙선거여론조사심의위원회,한길리서치,행정안전부","서구,김상훈,3선,무난,의정,활동,평가,대통령,국정수행,62.1%,30.4%,긍정,...","대구,김상훈,ars,한길리서치,홈페이지,행정안전부,응답자,3선,홍준표,부여,정의당,...",김상훈 국민의힘 의원 대구 서구 . 대구 ...,https://news.imaeil.com/page/view/202301152122...,NaN
4,9,1.500601e+06,20230115,매일신문,이수현,[대구 국회의원 평가 여론조사] 동구갑 류성걸 의원,지역>대구,지역>대전,정치>국회_정당,NaN,NaN,NaN,"류성걸,윤석열,홍준표","대구,대구시,동구,동구갑,부여","국민의힘,더불어민주당,매일신문,정의당,중앙선거여론조사심의위원회,한길리서치,행정안전부","의원,동구갑,류성걸,재선,평균치,의원,류성걸,국민의힘,전반적,의정,활동,주민들,대구...","동구갑,류성걸,대구,국민의힘,동구,ars,한길리서치,홈페이지,행정안전부,18세,지역...",류성걸 국민의힘 의원 대구 동구갑 . 류성...,https://news.imaeil.com/page/view/202301152115...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39941,80036,2.100311e+06,20230821,서울경제,서재원 기자,1년 전 銀을 金으로 우상혁 '위대한 도전',스포츠>올림픽_아시안게임,스포츠>축구,스포츠>월드컵,NaN,NaN,NaN,"김현섭,우상혁","독일,미국,부다페스트,도하,일본,유진 세계선수권대회,한국,세계선수권,뮌헨,텍사스,쿠...","아카마쓰,용인시청","1년,우상혁,도전,세계,선수권,높이뛰기,우승,도전,바르심,무타즈,에사,32,저본,해...","우상혁,한국,세계선수권,미국,해리슨,높이뛰기,선수권,바르심,런던,다이아몬드리그,유진...",우상혁이 20일 2023 부다페스트 세계육상선수권 남자 높이뛰기 예선에서 바를 넘고...,http://www.sedaily.com/NewsView/29THRF514Y,NaN
39942,80038,2.100311e+06,20230820,서울경제,여론독자부,[인사] 중소벤처기업부 외,지역>대구,경제>취업_창업,지역>전북,NaN,NaN,NaN,"이용주,이대희,박금철,김진명,이현조,조만희,이형철,김현곤,원영준,강영규,김언성,박정현","대구,경북지방중소벤처","△공공정책국장,△중앙노동위원회,기획재정부,△재정관리국장,고용노동부,중소벤처기업부","중소벤처기업부,전보,중소,벤처,기업부,실장,중소기업정책실장,전보,이대희,소상공인,정...","정책관,김현곤,박금철,조만희,이용주,중소벤처기업부,김언성,김진명,이형철,이대희,원영...",중소벤처기업부 실장급 전보 중소기업정책실장 이대희 소상공인정책실장 원영준 국...,http://www.sedaily.com/NewsView/29THBVO7B0,예외
39943,80040,2.100311e+06,20230820,서울경제,김경훈 기자,"'명예훼손 실형' 김용호, 조국 김건모 전처에 사과 ""깊이 반성""",사회>미디어,사회>사건_사고,NaN,NaN,NaN,NaN,"김,조,여배우,김건모,강용석,조국,김용호",대구,"유튜브,가로세로연구소,KNL,법무부,법원,유튜버","명예,훼손,실형,김용호,조국,김건모,전처,사과,반성,연예,조국,법무부,장관,가수,김...","김씨,김건모,김용호,사생활,장씨,연구소,가로세로,유튜브,가로세로연구소,유튜버,연예부...",유튜버 김용호씨. 유튜브 채널 강용석 나이트 라이브 캡처연예 출신 유튜...,http://www.sedaily.com/NewsView/29THC9PDCP,NaN
39944,80042,2.100311e+06,20230820,서울경제,이승령 기자,이러니 치안 불안할 수 밖에 현장 누빌 순경 1.4만명 부족,지역>광주,지역>부산,지역>울산,NaN,NaN,NaN,"순경,정우택","경기북,경기남부,서울,부산,대구,인천,경남,지구대","서울경찰청,행정안전위원회,국회,경찰청,경창청,국민의힘,비간부","치안,불안,현장,1.4,누빌,순경,1.4,부족,경감,경위,경정,간부직,정원,사건,시...","경찰관,경찰청,퇴직자,서울,시도경찰청,치안현장,경찰관들,정우택,3만,6급,국민의힘,...",윤희근 경찰청장이 18일 충북 충주시 중앙경찰학교에서 열린 신임경찰 제312기 졸업...,http://www.sedaily.com/NewsView/29THCKN45F,NaN


In [5]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer

In [6]:
def generate_summary(text):
    parser = PlaintextParser.from_string(text, Tokenizer("korean"))
    summarizer = LsaSummarizer()
    summary = summarizer(parser.document, sentences_count=2)
    return ' '.join([str(sentence) for sentence in summary])

# '요약' 컬럼을 생성하기 위해 tqdm을 사용한 반복문
tqdm.pandas()  # pandas와 함께 사용하기 위한 tqdm 설정

df['요약'] = df['본문'].progress_apply(generate_summary)

# 결과를 출력하거나 저장할 수 있습니다.
df.to_csv('output.csv')

 29%|█████████████████████▏                                                  | 11739/39946 [2:27:26<8:30:08,  1.09s/it]C:\Users\LG\anaconda3_2\envs\daegu\lib\site-packages\sumy\summarizers\lsa.py:76: UserWarning: Number of words (37) is lower than number of sentences (51). LSA algorithm may not work properly.
  warn(message % (words_count, sentences_count))
100%|██████████████████████████████████████████████████████████████████████████| 39946/39946 [7:03:38<00:00,  1.57it/s]


In [19]:
sample_df['본문'][0]

'교육적 회복 사라진 학폭위 궁극적 목적은 화해생기부 기재 부작용 커 가벼운 사안은 제외해야교권침해 학생부 기재도 신중히 접근해야 학부모 갈등 중재관  전국 최초 운영 계획   조희연 서울시교육감조희연 서울시교육감이 지난 10일 과 인터뷰에서 교육적 해결을 위한 학폭법 개정을 강조하고 있다. 안주영 전문지금 학교폭력 제도의 가장 큰 문제는  교육적 회복 이 사라졌다는 점이다. 법정화된 학교는 가해 학생의 처벌에만 집중했다. 그러는 사이 학생들의 관계 회복이나 피해 학생의 일상 회복은 더딜 수밖에 없었다. 은 최근  학폭위 10년  지금 우리 학교는  기획 기사를 통해 현행 학폭위 제도의 문제점과 실태를 고스란히 보도했다. 보도 이후 조희연 서울시교육감은 이 제기한 학폭위의 문제에 대해 적극 공감한다며  함께 대안을 찾아보자는 의견을 보내왔다. 그는 초등학교 저학년생을 학폭 제도에서 제외하고 경미한 사안은 학교생활기록부 기재를 아예 하지 않는 등 교육적 회복이 가능한 방안을 제시했다. 전국시도교육감협의회장인 조 교육감은 이런 방안들을 전국 교육감 합의를 거쳐 법 개정으로 이끌겠다고 강조했다. 인터뷰는 지난 10일 서울시교육청에서 대면으로 진행했다. 다음은 조 교육감과의 일문일답. 학폭 제도를 도입한 지 10년을 맞았다. 현장에서는 학폭 제도가 과연 우리 교실을 정말 행복하게 만들었는지에 대한 의문이 가득한 상황이다. 의 보도를 학폭 업무 담당자들과 인상 깊게 살펴봤다. 기사들이 제기한 문제의식에 매우 공감한다. 2011년 대구 중학생 집단 괴롭힘 사건을 계기로 학교폭력예방 및 대책에 관한 법률 학폭법 이 강화됐다. 심각한 학폭에서 피해 학생을 보호하고  가해 학생에 대한 징벌적 효과를 도입하는 취지다. 그런데 학생들 간의 갈등을 폭력이라는 범주로 다루다 보니 갈등 행위를 과도하게 엄중한 행위로 처리하기도 한다. 사소한 학폭도 무조건 신고 후 학교폭력대책심의위원회 학폭위 에서 처리한다. 학교 조치에 대해 법률적 흠결을 따지는 사례가 늘어나다 보니 이전처럼 학교에서